In [1]:
import pandas as pd
import numpy as np

In [93]:
df = pd.read_csv("dataset.csv")

C:\Users\HP\AppData\Local\Temp\ipykernel_6712\1745420125.py:1: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dataset.csv")


In [ ]:
import pandas as pd
import dabl  # Optional, for ML-based inference

def infer_and_convert_dtypes(df):
    """
    Infers column data types and converts them to the best possible types.
    Returns the cleaned DataFrame.
    """
    dtype_mapping = {}

    for col in df.columns:
        inferred = pd.api.types.infer_dtype(df[col])

        if inferred in ['integer', 'floating']:
            dtype_mapping[col] = df[col].dtype  # Keep as numeric
        elif inferred in ['boolean']:
            dtype_mapping[col] = 'boolean'
        elif inferred in ['datetime']:
            dtype_mapping[col] = 'datetime64[ns]'
        elif inferred in ['string', 'unicode']:
            dtype_mapping[col] = 'string'
        elif inferred in ['mixed', 'categorical']:
            dtype_mapping[col] = 'category'
        else:
            dtype_mapping[col] = 'object'  # Default to object

    # Apply inferred data types
    for col, dtype in dtype_mapping.items():
        try:
            df[col] = df[col].astype(dtype)
        except Exception as e:
            print(f"Warning: Couldn't convert {col} to {dtype}. Error: {e}")

    return df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   photoUrl          18979 non-null  string
 1   LongName          18979 non-null  string
 2   playerUrl         18979 non-null  string
 3   Nationality       18979 non-null  string
 4   Positions         18979 non-null  string
 5   Name              18979 non-null  string
 6   Age               18979 non-null  Int64 
 7   ↓OVA              18979 non-null  Int64 
 8   POT               18979 non-null  Int64 
 9   Team & Contract   18979 non-null  string
 10  ID                18979 non-null  Int64 
 11  Height            18979 non-null  string
 12  Weight            18979 non-null  string
 13  foot              18979 non-null  string
 14  BOV               18979 non-null  Int64 
 15  BP                18979 non-null  string
 16  Growth            18979 non-null  Int64 
 17  Joined      

In [113]:
import pandas as pd
import dabl  # Auto-infers column types

df = pd.read_csv("dataset.csv")
df_cleaned = dabl.detect_types(df)  # Automatically detects column types

df_cleaned # Shows inferred types


C:\Users\HP\AppData\Local\Temp\ipykernel_6712\1026067717.py:4: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dataset.csv")
c:\Users\HP\OneDrive\Documents\PurifAI\purifai\Lib\site-packages\dabl\preprocessing.py:177: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(series[:10])
c:\Users\HP\OneDrive\Documents\PurifAI\purifai\Lib\site-packages\dabl\preprocessing.py:177: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(series[:10])
c:\Users\HP\OneDrive\Documents\PurifAI\purifai\Lib\site-packages\dabl\preprocessing.py:177: UserWarning: Could not infer format, so each element will be parsed individually, falling back to 

,continuous,dirty_float,low_card_int_ordinal,low_card_int_categorical,categorical,date,free_string,useless
photoUrl,False,False,False,False,False,False,True,False
LongName,False,False,False,False,False,False,True,False
playerUrl,False,False,False,False,False,False,True,False
Nationality,False,False,False,False,True,False,False,False
Positions,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...
PAS,False,False,True,False,False,False,False,False
DRI,False,False,True,False,False,False,False,False
DEF,False,False,True,False,False,False,False,False
PHY,False,False,True,False,False,False,False,False


In [117]:
import pandas as pd

# Load dataset safely
df = pd.read_csv("dataset.csv", low_memory=False)

# Step 1: Ensure numeric columns are properly detected
df = df.convert_dtypes()

# Step 2: Explicitly handle datetime columns
for col in df.columns:
    try:
        # Convert columns that look like dates
        df[col] = pd.to_datetime(df[col], format="%Y-%m-%d", errors="coerce")
    except:
        pass

# Step 3: Check final column types
print(df.dtypes)


photoUrl       datetime64[ns]
LongName       datetime64[ns]
playerUrl      datetime64[ns]
Nationality    datetime64[ns]
Positions      datetime64[ns]
                    ...      
PAS            datetime64[ns]
DRI            datetime64[ns]
DEF            datetime64[ns]
PHY            datetime64[ns]
Hits           datetime64[ns]
Length: 77, dtype: object


In [118]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from scipy import stats

def clean_and_engineer_data(file_path):
    """ Fully automates data cleaning & feature engineering, keeping only original columns """

    # Load data
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    elif file_path.endswith('.xlsx'):
        df = pd.read_excel(file_path)
    elif file_path.endswith('.json'):
        df = pd.read_json(file_path)
    else:
        raise ValueError("Unsupported file format. Use CSV, Excel, or JSON.")

    # Backup original column names
    original_columns = df.columns.tolist()

    # Drop duplicates
    df = df.drop_duplicates()

    # Handle missing values (numeric → mean, categorical → most frequent)
    for col in df.columns:
        if df[col].dtype == 'object':  # Categorical
            df[col] = df[col].fillna(df[col].mode()[0])
        else:  # Numeric
            df[col] = df[col].fillna(df[col].mean())

    # Remove outliers using IQR
    def remove_outliers(df):
        num_cols = df.select_dtypes(include=np.number).columns
        for col in num_cols:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            df = df[(df[col] >= Q1 - 1.5 * IQR) & (df[col] <= Q3 + 1.5 * IQR)]
        return df

    df = remove_outliers(df)

    # Encode categorical variables (but keep original format)
    encoder = OneHotEncoder(sparse_output=False, drop='first')
    categorical_cols = df.select_dtypes(include=['object']).columns

    if len(categorical_cols) > 0:
        encoded_array = encoder.fit_transform(df[categorical_cols])
        encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(categorical_cols))
        
        # Keep only the original categorical columns, replacing them with their encoded versions
        df = df.drop(columns=categorical_cols)
        df[encoded_df.columns] = encoded_df

    # Scale numerical columns if needed
    num_cols = df.select_dtypes(include=np.number).columns
    scaler = StandardScaler()
    df[num_cols] = scaler.fit_transform(df[num_cols])

    # Restore original column names
    df.columns = original_columns

    return df

# Example usage
cleaned_df = clean_and_engineer_data("dataset.csv")
print(cleaned_df.head())


C:\Users\HP\AppData\Local\Temp\ipykernel_6712\2368985367.py:12: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


TypeError: Encoders require their input argument must be uniformly strings or numbers. Got ['int', 'str']

In [127]:
import pandas as pd
import numpy as np
df=pd.read_csv("dataset.csv")
def clean_data(file_path):
    """Cleans messy dataset: fixes datatypes, fills missing values, removes outliers."""
    
    # Load dataset efficiently
    try:
        df = pd.read_csv(file_path, low_memory=False)
    except Exception as e:
        print(f"Error loading file: {e}")
        return None

    # Drop duplicates & completely empty columns
    df = df.drop_duplicates()
    df = df.dropna(axis=1, how='all')

    # Convert column names to lowercase & remove leading/trailing spaces
    df.columns = df.columns.str.lower().str.strip()

    # Detect and fix incorrect data types
    for col in df.columns:
        df[col] = df[col].apply(lambda x: np.nan if str(x).strip().lower() in ["nan", "none", "null", "n/a", ""] else x)

        if df[col].dtype == 'object':  # If it's a string column
            try:
                df[col] = pd.to_numeric(df[col])  # Try converting to numeric
            except:
                pass  # If it fails, keep it as a categorical/text column

    # Fill missing values intelligently
    for col in df.columns:
        if df[col].dtype in ['int64', 'float64']:  # Numeric columns
            df[col] = df[col].fillna(df[col].median())  # Use median for robustness
        else:  # Categorical columns
            df[col] = df[col].fillna(df[col].mode()[0])  # Use most common value

    # Handle outliers using IQR method (more robust than standard deviation)
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    for col in numeric_cols:
        Q1, Q3 = df[col].quantile(0.25), df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound, upper_bound = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

    # Convert categorical columns to proper string type
    cat_cols = df.select_dtypes(include=['object']).columns
    df[cat_cols] = df[cat_cols].astype(str)

    return df

# Example usage
cleaned_df = clean_data("dataset.csv")
print(cleaned_df.info())  # Check final datatypes
 # Preview cleaned data


C:\Users\HP\AppData\Local\Temp\ipykernel_6712\547523651.py:3: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("dataset.csv")


<class 'pandas.core.frame.DataFrame'>
Index: 12564 entries, 146 to 18962
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   photourl          12564 non-null  object
 1   longname          12564 non-null  object
 2   playerurl         12564 non-null  object
 3   nationality       12564 non-null  object
 4   positions         12564 non-null  object
 5   name              12564 non-null  object
 6   age               12564 non-null  int64 
 7   ↓ova              12564 non-null  int64 
 8   pot               12564 non-null  int64 
 9   team & contract   12564 non-null  object
 10  id                12564 non-null  int64 
 11  height            12564 non-null  object
 12  weight            12564 non-null  object
 13  foot              12564 non-null  object
 14  bov               12564 non-null  int64 
 15  bp                12564 non-null  object
 16  growth            12564 non-null  int64 
 17  joined         

In [132]:
print(f"DFSHAPE: {df.shape}")
print(f"NEWDF SHAPE: {cleaned_df.shape}")

DFSHAPE: (18979, 77)
NEWDF SHAPE: (12564, 77)


In [133]:
import pandas as pd
import numpy as np
import re
from scipy.stats.mstats import winsorize

def data_standardization(df):
    """
    Standardizes the cleaned dataframe by performing:
    - Column name formatting
    - String standardization (trim spaces, lowercase)
    - Duplicate removal
    - Handling incorrect values
    - Converting data types (dates, numerics, categories)
    - Feature engineering & outlier handling
    """

    # 1️⃣ Standardize Column Names
    df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')

    # 2️⃣ Trim Spaces & Convert Strings to Lowercase
    df = df.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)

    # 3️⃣ Convert Dates to Standard Format
    for col in df.select_dtypes(include=['object']).columns:
        try:
            df[col] = pd.to_datetime(df[col], errors='coerce')  # Convert date columns
        except:
            pass

    # 4️⃣ Handle Missing Values
    for col in df.columns:
        if df[col].isnull().sum() / len(df) > 0.8:  # Drop columns with >80% missing
            df.drop(columns=[col], inplace=True)
        elif df[col].dtype == 'object':
            df[col].fillna(df[col].mode()[0], inplace=True)  # Fill categorical with mode
        else:
            df[col].fillna(df[col].median(), inplace=True)  # Fill numerical with median

    # 5️⃣ Remove Duplicates
    df.drop_duplicates(inplace=True)

    # 6️⃣ Fix Inconsistent Categorical Values
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)  # Remove special chars

    # 7️⃣ Handle Outliers with Winsorization
    for col in df.select_dtypes(include=['int64', 'float64']).columns:
        df[col] = winsorize(df[col], limits=[0.05, 0.05])  # Clip extreme 5% values

    # 8️⃣ Feature Engineering: Extract Year if Date Exists
    date_cols = df.select_dtypes(include=['datetime64']).columns
    for col in date_cols:
        df[f"{col}_year"] = df[col].dt.year

    return df


In [135]:
standardized_data = data_standardization(cleaned_df)

C:\Users\HP\AppData\Local\Temp\ipykernel_6712\2088059554.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)
C:\Users\HP\AppData\Local\Temp\ipykernel_6712\2088059554.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')  # Convert date columns
C:\Users\HP\AppData\Local\Temp\ipykernel_6712\2088059554.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')  # Convert date columns
C:\Users\HP\AppData\Local\Temp\ipykernel_6712\2088059554.py:26: UserWarning: Could not infer format, so each element will be pa

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

def clean_and_standardize_data(file_path):
    # Load dataset
    df = pd.read_csv(file_path)
    
    # Remove duplicate rows
    df = df.drop_duplicates()
    
    # Replace string "nan" with actual NaN values
    df.replace("nan", np.nan, inplace=True)
    
    # Handling missing values
    for col in df.columns:
        if df[col].isna().sum() / len(df) > 0.6:
            df.drop(columns=[col], inplace=True)  # Drop columns with more than 60% missing values
        elif df[col].dtype in ['int64', 'float64']:
            imputer = SimpleImputer(strategy='median')  # Fill numeric columns with median
            df[col] = imputer.fit_transform(df[[col]])
        else:
            imputer = SimpleImputer(strategy='most_frequent')  # Fill categorical columns with mode
            df[col] = imputer.fit_transform(df[[col]])
    
    # Drop low variance columns (almost all values the same)
    for col in df.select_dtypes(include=['int64', 'float64']).columns:
        if df[col].nunique() / len(df) < 0.01:
            df.drop(columns=[col], inplace=True)
    
    # Drop highly correlated columns (above 0.95 correlation)
    corr_matrix = df.corr()
    upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.95)]
    df.drop(columns=to_drop, inplace=True)
    
    # Standardization (applies only to numeric columns)
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    scaler = StandardScaler()
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
    
    return df


(12564, 59)

In [143]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import StandardScaler

def remove_special_chars(text):
    """Removes special symbols, emojis, and unnecessary characters from text."""
    if isinstance(text, str):
        return re.sub(r"[^\w\s]", "", text)  # Keeps only letters, numbers, and spaces
    return text

def clean_and_standardize_data(file_path, corr_threshold=0.9):
    """
    Cleans messy dataset: fixes datatypes, fills missing values, removes outliers,
    drops highly correlated columns, removes special symbols, and standardizes numerical features.
    """
    
    # Load dataset
    try:
        df = pd.read_csv(file_path, low_memory=False)
    except Exception as e:
        print(f"Error loading file: {e}")
        return None

    # Drop duplicates & completely empty columns
    df = df.drop_duplicates()
    df = df.dropna(axis=1, how='all')

    # Convert column names to lowercase & remove leading/trailing spaces
    df.columns = df.columns.str.lower().str.strip()

    # Detect and fix incorrect data types
    for col in df.columns:
        df[col] = df[col].apply(lambda x: np.nan if str(x).strip().lower() in ["nan", "none", "null", "n/a", ""] else x)

        if df[col].dtype == 'object':  # If it's a string column
            try:
                df[col] = pd.to_numeric(df[col])  # Try converting to numeric
            except:
                pass  # If it fails, keep it as a categorical/text column

    # Remove special symbols & emojis from text columns
    text_cols = df.select_dtypes(include=['object']).columns
    df[text_cols] = df[text_cols].applymap(remove_special_chars)

    # Fill missing values intelligently
    for col in df.columns:
        if df[col].dtype in ['int64', 'float64']:  # Numeric columns
            df[col] = df[col].fillna(df[col].median())  # Use median for robustness
        else:  # Categorical columns
            df[col] = df[col].fillna(df[col].mode()[0])  # Use most common value

    # Handle outliers using IQR method
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    for col in numeric_cols:
        Q1, Q3 = df[col].quantile(0.25), df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound, upper_bound = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

    # Drop highly correlated features
    corr_matrix = df[numeric_cols].corr().abs()
    upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    drop_cols = [column for column in upper_triangle.columns if any(upper_triangle[column] > corr_threshold)]
    df = df.drop(columns=drop_cols)

    # ✅ Recalculate numeric_cols after dropping correlated features
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

    # Standardize numerical columns
    scaler = StandardScaler()
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

    return df


# Example usage
file_path = "dataset.csv"
cleaned_standardized_df = clean_and_standardize_data(file_path)

print(cleaned_standardized_df.head())


C:\Users\HP\AppData\Local\Temp\ipykernel_6712\3616696370.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[text_cols] = df[text_cols].applymap(remove_special_chars)


                                   photourl                       longname  \
146  httpscdnsofifacomplayers23048121_60png  Ronaldo Jailson Cabrais Petri   
150  httpscdnsofifacomplayers22792821_60png           Nélson Cabral Semedo   
158  httpscdnsofifacomplayers21969321_60png      Diego Carlos Santos Silva   
161  httpscdnsofifacomplayers21533321_60png                   Duván Zapata   
165  httpscdnsofifacomplayers21051421_60png      João Pedro Cavaco Cancelo   

                                             playerurl nationality positions  \
146  httpsofifacomplayer230481ronaldojailsoncabrais...      Brazil    CAM RM   
150  httpsofifacomplayer227928nelsoncabralsemedo210005    Portugal        RB   
158  httpsofifacomplayer219693diegocarlossantossilv...      Brazil        CB   
161         httpsofifacomplayer215333duvanzapata210005    Colombia        ST   
165  httpsofifacomplayer210514joaopedrocavacocancel...    Portugal        RB   

                name       age      ↓ova       pot

In [144]:
cleaned_standardized_df

,photourl,longname,playerurl,nationality,positions,name,age,↓ova,pot,team & contract,...,gk kicking,gk positioning,gk reflexes,w/f,sm,a/w,d/w,ir,phy,hits
146,httpscdnsofifacomplayers23048121_60png,Ronaldo Jailson Cabrais Petri,httpsofifacomplayer230481ronaldojailsoncabrais...,Brazil,CAM RM,Ronaldo Cabrais,0.707271,2.695886,1.991996,\n\n\n\nGrêmio\n2019 2023\n\n,...,0.855919,1.529848,0.875983,4,4,Medium,Low,1,0.834725,\n46
150,httpscdnsofifacomplayers22792821_60png,Nélson Cabral Semedo,httpsofifacomplayer227928nelsoncabralsemedo210005,Portugal,RB,Nélson Semedo,0.240157,2.695886,2.512848,\n\n\n\nWolverhampton Wanderers\n2020 2023\n\n,...,-0.145580,1.194438,0.539361,3,4,Medium,High,2,1.286938,\n137
158,httpscdnsofifacomplayers21969321_60png,Diego Carlos Santos Silva,httpsofifacomplayer219693diegocarlossantossilv...,Brazil,CB,Diego Carlos,0.473714,2.695886,2.339230,\n\n\n\nSevilla FC\n2019 2024\n\n,...,1.189752,-1.153433,0.202739,4,3,Medium,High,2,2.078310,\n170
161,httpscdnsofifacomplayers21533321_60png,Duván Zapata,httpsofifacomplayer215333duvanzapata210005,Colombia,ST,D Zapata,0.940828,2.695886,1.991996,\n\n\n\nAtalanta\n2020 2023\n\n,...,-0.813247,-1.488843,-0.807126,4,3,High,Medium,2,2.078310,\n123
165,httpscdnsofifacomplayers21051421_60png,João Pedro Cavaco Cancelo,httpsofifacomplayer210514joaopedrocavacocancel...,Portugal,RB,João Cancelo,0.240157,2.695886,2.512848,\n\n\n\nManchester City\n2019 2025\n\n,...,1.523585,1.194438,1.212604,3,4,High,Medium,2,0.608619,\n117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18945,httpscdnsofifacomplayers25255621_60png,Brendon Shabani,httpsofifacomplayer252556brendonshabani210005,Albania,CM,B Shabani,-1.628302,-2.942721,-0.612262,\n\n\n\nLeyton Orient\n2019 2021\n\n,...,1.523585,1.194438,0.875983,3,2,Medium,Medium,1,-1.765497,\n5
18950,httpscdnsofifacomplayers24772021_60png,Chaoyang Liu,httpsofifacomplayer247720chaoyangliu210005,China PR,CM,Liu Chaoyang,-0.927630,-2.942721,-2.001199,\n\n\n\nShandong Luneng TaiShan FC\n2019 2022...,...,1.189752,0.859028,0.202739,3,2,Medium,Medium,1,-0.974125,\n2
18959,httpscdnsofifacomplayers25826821_60png,Ethan Espinoza,httpsofifacomplayer258268ethanespinoza210005,Chile,CM,E Espinoza,-1.394745,-2.942721,-1.827582,\n\n\n\nColoColo\n2020 2024\n\n,...,-0.145580,1.194438,1.549226,2,2,Medium,Medium,1,-0.748019,\n2
18960,httpscdnsofifacomplayers25111021_60png,Haijian Wang,httpsofifacomplayer251110haijianwang210005,China PR,CM,Wang Haijian,-1.394745,-2.942721,-2.522051,\n\n\n\nShanghai Greenland Shenhua FC\n2019 2...,...,0.855919,0.859028,0.875983,2,2,Medium,Medium,1,-1.313285,\n1
